<a href="https://colab.research.google.com/github/FelipeM0reira/analise_vitrine_da_moda/blob/main/analise_vitrine_da_moda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('vendas.csv', encoding='latin-1', sep=';')

In [ ]:
print(df)

            Data           Marca        Produto Gênero     Preço     \
0     31/08/2023          Adidas  Calça legging      F   R$ 131,94    
1     31/08/2023         DiPaula  Calça legging      F    R$ 35,90    
2     31/08/2023         DiPaula  Calça legging      F    R$ 35,90    
3     31/08/2023   Extra Fitness  Calça legging      F    R$ 29,90    
4     31/08/2023   Extra Fitness  Calça legging      F    R$ 29,90    
...          ...             ...            ...    ...          ...   
2080  31/07/2024            Nike        Vestido      F   R$ 274,99    
2081  31/07/2024            Nike        Vestido      F   R$ 274,99    
2082  31/07/2024            Nike        Vestido      F   R$ 274,99    
2083  31/07/2024  Sukira Fitness        Vestido      F    R$ 69,90    
2084  31/07/2024  Sukira Fitness        Vestido      F    R$ 69,90    

      Quantidade     Total     
0              2   R$ 263,88   
1              3   R$ 107,70   
2              3   R$ 107,70   
3              1   